In [1]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
from pathlib import Path
SEED = 42
np.random.seed(SEED)

ModuleNotFoundError: No module named 'cv2'

## Visualizing Image From Dataset

In [ ]:
class_names=["SignBoard"]

colors = np.random.uniform(0, 255, size=(len(class_names), 3))

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)
        width = xmax - xmin
        height = ymax - ymin

        class_name = class_names[int(labels[box_num])]

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=colors[class_names.index(class_name)],
            thickness=3
        )

        font_scale = min(1,max(3,int(w/500)))
        font_thickness = min(2, max(10,int(w/50)))

        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
        # Text width and height
        tw, th = cv2.getTextSize(
            class_name,
            0, fontScale=font_scale, thickness=font_thickness
        )[0]
        p2 = p1[0] + tw, p1[1] + -th - 10
        cv2.rectangle(
            image,
            p1, p2,
            color=colors[class_names.index(class_name)],
            thickness=-1,
        )
        cv2.putText(
            image,
            class_name,
            (xmin+1, ymin-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            font_thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_training_images = glob.glob(image_paths)
    all_training_labels = glob.glob(label_paths)
    all_training_images.sort()
    all_training_labels.sort()

    num_images = len(all_training_images)

    plt.figure(figsize=(9, 7))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image = cv2.imread(all_training_images[j])
        with open(all_training_labels[j], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=0)
    plt.tight_layout()
    plt.show()

In [ ]:
plot(
    image_paths='/kaggle/input/signboard-detection-using-yolo-v8/train/images/*',
    label_paths='/kaggle/input/signboard-detection-using-yolo-v8/train/labels/*',
    num_samples=4,
)

ValueError: empty range for randrange() (0, 0, 0)

<Figure size 900x700 with 0 Axes>

In [ ]:
%%writefile data.yaml
train: /kaggle/input/signboard-detection-dataset/train
val: /kaggle/input/signboard-detection-dataset/valid
test: /kaggle/input/signboard-detection-dataset/test

nc: 1

# class names
names:
  0: 'SignBoard'


In [ ]:
def monitor_tensorboard():
    %load_ext tensorboard
    %tensorboard --logdir runs/detect/

## Installing Yolo V8

In [ ]:
!pip3 install ultralytics

In [ ]:
#Importing Yolo
from ultralytics import YOLO

# Create a new YOLO model from scratch
model = YOLO('yolov8m.yaml')

# Train the model using the 'coco128.yaml' dataset for 3 epochs
results = model.train(data='data.yaml', epochs=50, batch=32, imgsz=640, save=True, save_period=1,  pretrained=True,optimizer="Adam",lr0=0.0001,lrf=0.01, val=True, verbose=True)

success = model.export()


In [ ]:
# Evaluating our model
model = YOLO('runs/detect/train/weights/best.pt')
results = model.val(data='data.yaml',imgsz=720,batch=1,conf=0.4,iou=0.5)

In [ ]:
monitor_tensorboard()

In [ ]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):

    plt.figure(figsize=(20, 12))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [ ]:
#Predicting On testing Dataset

model = YOLO('results/weighths/best.pt')

results = model(source='/kaggle/input/signboard-detection-using-yolo-v8/test/images',imgsz=640,conf=0.3, iou=0.55,save=True)

